In [3]:
import tensorflow as tf

In [4]:
tf.test.is_gpu_available()

True

In [5]:
#%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print(device_name)
if device_name != '/device:DML:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

/device:DML:0
Found GPU at: /device:DML:0


In [6]:
tf.compat.v1.config.experimental.set_visible_devices

<function tensorflow.python.framework.config.set_visible_devices(devices, device_type=None)>

In [35]:
import tensorflow as tf
import numpy as np
from tensorflow.python.keras.utils.vis_utils import plot_model
import math
from tensorflow.keras.layers import Layer
from tensorflow import keras

In [36]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


In [37]:
from tensorflow.python.ops.gradients_impl import gradients
class IPC(keras.Model):

    def __init__(self, filters,k,alpha):
        '''Initializes the class and sets up the internal variables'''
        # YOUR CODE HERE

        super(IPC,self).__init__()
        print("starting the  Improve convolution block")
        #self.inputs=inputs
        #self.input_shap=self.inputs.shape
        self.filters=filters
        self.filters1=int(self.filters*alpha)
        self.filters2=int(self.filters*(1-alpha))
        self.k=k
        self.alpha=alpha
        self.z_i=None
        self.seed=None
        self.activated_beta=None
        self.input_shap=(8,8,512)
    
    def build(self,inputs):
        '''Create the state of the layer (weights)'''
        self.pointwise1=tf.keras.layers.Conv2D(self.filters2,1,strides=1,use_bias=False)
        seed_init=tf.random_uniform_initializer(minval=-1, maxval=0)
        seed_init_val=seed_init(shape=(50,1,self.input_shap[-1],1),dtype='float32')
        """
        need to initialize a seed with shape (batch size,1,512,1) 
        """
     
        self.seed=tf.Variable(initial_value=seed_init_val,trainable=True)
        self.seed=tf.placeholder_with_default(self.seed,[None]+[i for i in self.seed.shape[1:]])
        self.z_i=self.seed*self.alpha
        self.pointwise2=tf.keras.layers.Conv2D(self.filters1,1,strides=1,use_bias=False)
        self.convbeta=tf.keras.layers.Conv2D(self.k,(1,self.filters1+1),strides=1,padding='valid',use_bias=False)
        
   
    def call(self,inputs):
        '''Defines the computation from inputs to outputs'''
        inpu_shape=inputs.shape

        with tf.GradientTape() as tape:
          beta=self.convbeta(self.z_i)

          self.activated_beta=tf.math.tanh(beta)
        pw1=self.pointwise1(inputs)
        pw1=tf.keras.layers.BatchNormalization()(pw1)
        pw1=tf.keras.layers.Activation('tanh')(pw1)
        shape1=pw1.shape
        pw2=self.pointwise2(self.activated_beta)
        pw2=tf.keras.layers.BatchNormalization()(pw2)
        pw2=tf.keras.layers.Activation('tanh')(pw2)

        shape=pw2.shape
        pw2_reshape=tf.reshape(pw2,(-1,inpu_shape[1],inpu_shape[2]
                                    ,(shape[1]*shape[2]*shape[3])//(inpu_shape[1]*inpu_shape[2])))
       
        return tf.keras.layers.Concatenate(axis=3)([pw1, pw2_reshape])


    def train_step(self, inputs,targets):
        #inputs, targets = data
        with tf.GradientTape() as tape:
            predictions = self(inputs, training=True)
            loss = self.compiled_loss(targets, predictions)               
        gradients = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_weights))
        self.compiled_metrics.update_state(targets, predictions) 
        #print(m)
        return {m.name: m.result() for m in self.metrics}        

In [38]:
input=tf.keras.layers.Input((8,8,512) )
x1 = tf.keras.layers.DepthwiseConv2D(3, strides=1, padding='same',use_bias=False)(input)
n,h,w,c=x1.shape
print(n,h,w,c)
a=IPC(512,9,0.875)
outputs=a(x1)
#a.train_step(x1,outputs)
#print(f'output{a(x1).shape}')
model= tf.keras.models.Model(input, outputs=outputs)
model.summary()

? 8 8 512
starting the  Improve convolution block
Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 8, 8, 512)]       0         
_________________________________________________________________
depthwise_conv2d_74 (Depthwi (None, 8, 8, 512)         4608      
_________________________________________________________________
ipc_39 (IPC)                 (None, 8, 8, 512)         40841     
Total params: 45,449
Trainable params: 45,449
Non-trainable params: 0
_________________________________________________________________


In [40]:
def mobilenet_block(x, f, s=1):
    x = tf.keras.layers.DepthwiseConv2D(3, strides=s, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    x = tf.keras.layers.Conv2D(f, 1, strides=1, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x =tf.keras.layers.Activation('relu')(x)
    return x

In [41]:

def ManishNet(input_shape, n_classes):

    """
    Manish's variation of MobileNetV1
    """

    input = tf.keras.layers.Input(input_shape)
    x = tf.keras.layers.Conv2D(32, 3, strides=1, padding='same')(input)
    #print("conv2d --1",x.shape)
    x = tf.keras.layers.BatchNormalization()(x)
    #print("batchNormalzation",x.shape)
    x = tf.keras.layers.Activation('relu')(x)
    #print("activation",x.shape)
    x = mobilenet_block(x, 64)
    x = mobilenet_block(x, 64,2)
    #print("1st mobilenetblock_1",x.shape)
    x = mobilenet_block(x, 128, 1)
    #print("2nd mobilenetblock_2",x.shape)
    x = mobilenet_block(x, 128)
    #print("3rd mobilenetblock_3",x.shape)

    x = mobilenet_block(x, 256)
    #print("4th mobilenetblock_4",x.shape)
    x = mobilenet_block(x, 256,2)
    #print("5th mobilenet_block_5",x.shape)

    x = mobilenet_block(x, 512, 1)


    #print("6th mobilenetblock_6",x.shape)
    for i in range(5):

        x = tf.keras.layers.DepthwiseConv2D(3, strides=1, padding='same',use_bias=False)(x)
        a = IPC(512,alpha=0.875,k=9)
        x=a(x)
    # print("jjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjj",x.shape)
    #print(f"{i+7} Improvedblock shape {x.shape}")


    x = tf.keras.layers.DepthwiseConv2D(3, strides=1, padding='same',use_bias=False)(x)
    a = IPC(512,alpha=0.875,k=9)(x)
    # x=a(x)
    #print("12 IPC",x.shape)

    x = tf.keras.layers.DepthwiseConv2D(3, strides=1, padding='same',use_bias=False)(x)
    a = IPC(512,alpha=0.875,k=9)(x)
    #x=a(x)
    print("13 IPC",x.shape)

    x = tf.keras.layers.GlobalAvgPool2D()(x)
    # x = tf.keras.layers.AveragePooling2D()(x)

    output = tf.keras.layers.Dense(n_classes, activation='softmax')(x)
    print(output.shape)

    model = tf.keras.models.Model(input, output)
    return model


In [42]:
input_shape=32,32,3
n_classes=10
model=ManishNet(input_shape, n_classes)
model.summary()

starting the  Improve convolution block
starting the  Improve convolution block
starting the  Improve convolution block
starting the  Improve convolution block
starting the  Improve convolution block
starting the  Improve convolution block
starting the  Improve convolution block
13 IPC (?, 8, 8, 512)
(?, 10)
Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_161 (Conv2D)          (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_110 (Bat (None, 32, 32, 32)        128       
_________________________________________________________________
activation_110 (Activation)  (None, 32, 32, 32)        0         
_________________________________________________________________
depthwise_con

In [31]:
input_shape=32,32,3
n_classes=10
model=ManishNet(input_shape, n_classes)
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train=x_train.astype('float32')
x_train/=255
x_test=x_test.astype('float32')
x_test/=255
print(x_train.shape)
model.compile( loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam( learning_rate=0.001 ) , metrics=[ 'acc' ] )

starting the  Improve convolution block
pointwise1(?, 8, 8, 64)
after applying ptwise (?, 1, 64, 448)
after reshaping (?, 8, 8, 448)
starting the  Improve convolution block
pointwise1(?, 8, 8, 64)
after applying ptwise (?, 1, 64, 448)
after reshaping (?, 8, 8, 448)
starting the  Improve convolution block
pointwise1(?, 8, 8, 64)
after applying ptwise (?, 1, 64, 448)
after reshaping (?, 8, 8, 448)
starting the  Improve convolution block
pointwise1(?, 8, 8, 64)
after applying ptwise (?, 1, 64, 448)
after reshaping (?, 8, 8, 448)
starting the  Improve convolution block
pointwise1(?, 8, 8, 64)
after applying ptwise (?, 1, 64, 448)
after reshaping (?, 8, 8, 448)
starting the  Improve convolution block
pointwise1(?, 8, 8, 64)
after applying ptwise (?, 1, 64, 448)
after reshaping (?, 8, 8, 448)
starting the  Improve convolution block
pointwise1(?, 8, 8, 64)
after applying ptwise (?, 1, 64, 448)
after reshaping (?, 8, 8, 448)
13 IPC (?, 8, 8, 512)
(?, 10)
(50000, 32, 32, 3)


In [32]:
model.fit(x_train,y_train,shuffle=True,batch_size=50,epochs=2)

Train on 50000 samples
Epoch 1/2
50000/50000 [==============================] - 46s 930us/sample - loss: 1.7503 - acc: 0.3053
Epoch 2/2
50000/50000 [==============================] - 45s 903us/sample - loss: 1.2949 - acc: 0.5247
